## Assesing how the model compares with clinician rater on CLSA data

This file is part of the Glaucoma Phenotype ML Estimation project.

 Glaucoma Phenotype ML Estimation is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.


The Glaucoma Phenotype ML Estimation project is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with the Glaucoma Phenotype ML Estimation project.  If not, see <http://www.gnu.org/licenses/>.


Some data from the CLSA was graded by a clinican to compare to the models performance

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [5]:
import pickle as pkl
from pathlib import Path
from multiprocessing import Pool, cpu_count
import numpy as np

import pandas as pd
import zipfile
#import imageio
import matplotlib.pyplot as plt
import seaborn as sns
from fastai.vision import *
from IPython.display import display

from multiprocessing import freeze_support
from multiprocessing import Pool
from multiprocessing import cpu_count
from functools import partial
import threading

from fastai.vision import *
from fastai.vision.models import *
from fastai.vision.learner import model_meta
from fastai.callbacks import *
from PIL import Image
from glaucoma.helpers.glaucoma_helpers import *

In [6]:
##### PLEASE SET AS REQUIRED########
WORKING_DIR = Path(os.getcwd())
DATA_DIR = WORKING_DIR / 'data'
META_DIR = DATA_DIR / 'metadata'
TRAIN_DIR = DATA_DIR / 'train'
UKBB_DIR = DATA_DIR 'retinal_images/UKBB'
CROP_DIR = DATA_DIR /'retinal_images/cropped_UKBB')
C_CLSA_DIR = DATA_DIR / 'retinal_images/cropped_CLSA')
GRADE_DIR = DATA_DIR / "gradable"
CLSA_DIR =DATA_DIR / 'retinal_images/CLSA')

In [11]:
CLSA_preds = pd.read_csv(WORKING_DIR /'inference/CLSA_full_inference_v2.csv')

In [13]:
CLSA_clin = pd.read_csv('/home/jovyan/glaucoma/set_xaa_alexhewitt.VCDR',sep = '\t',header = None)


In [14]:
CLSA_clin.head(3)

,0,1
0,190225_QIMR_SMacGregor_1000258_left.jpg,3.5
1,190225_QIMR_SMacGregor_1000551_left.jpg,3
2,190225_QIMR_SMacGregor_1000651_left.jpg,2.5


In [16]:
# pull out relevant preds:
files = CLSA_clin[0]
ratings  =CLSA_clin[1]

In [17]:
used_preds = CLSA_preds[CLSA_preds['file_name'].isin(files)]
used_preds = used_preds[used_preds['follow_up'] == 1]

In [ ]:
# getting error between clincian and model
error = []
mse = []
error_file = {}
X =[]
Y =[]
gradabillity ={}
for i in files:
    if CLSA_clin[CLSA_clin[0] == i][1].item() != 'X':
        e = np.abs(used_preds[used_preds['file_name'] == i]['vcdr_estimate'].item() - float(CLSA_clin[CLSA_clin[0] == i][1].item()))
        error.append(e)
        mse.append(np.power(used_preds[used_preds['file_name'] == i]['vcdr_estimate'].item() - float(CLSA_clin[CLSA_clin[0] == i][1].item()),2))
        X.append(used_preds[used_preds['file_name'] == i]['vcdr_estimate'].item())
        Y.append(float(CLSA_clin[CLSA_clin[0] == i][1].item()))
    else:
        gradabillity[i] = used_preds[used_preds['file_name'] == i]['gradable'].item()
        
X = np.asarray(X)
Y = np.asarray(Y)

In [19]:
# root mean square error
np.sqrt(np.average(mse))

1.0991911148179758

In [21]:
correlation_matrix = np.corrcoef(X, Y)
correlation_xy = correlation_matrix[0,1]
r_squared = correlation_xy**2

In [22]:
#correlation between clinican and modek
correlation_xy

0.8394766459101302

In [23]:
ff = parse_files(C_CLSA_DIR)

In [72]:
f_name ='190225_QIMR_SMacGregor_1266754_left.jpg'
im = [i for i in ff if f_name in i and 'followup' in i]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#img=mpimg.imread(ukbb_images_right[0])
im = Image.open(im[0])
imgplot = plt.imshow(im)
plt.show()

In [79]:
from sklearn.metrics import accuracy_score
#from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

In [49]:
y_hat, y= categorical_convert_metrics([torch.tensor(X).view([-1,1]),torch.tensor(Y).view([1,-1]).t()])

accuracy: 0.2510593220338983


In [66]:
y_hat = X

In [77]:
# approx rating
pm_thres = 2.01
correct = 0
for i,truth in enumerate(Y):
    if y_hat[i] > truth - pm_thres and y_hat[i] < truth + pm_thres:
        correct +=1

In [78]:
#percent within 2 grading points of CLSA model
correct/944

0.9290254237288136